### UMAP

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import numba
import umap
import re
from scipy import stats
from scipy.stats import mstats
from scikit_posthocs import posthoc_dunn
pd.set_option('display.max_rows', 50)
np.set_printoptions(threshold=50)

In [ ]:
diagkeys = ['phenotype']

In [ ]:
%run -i setup_functions.py

In [ ]:
basedir = os.getcwd()

# Prepare Data

In [ ]:
## Get Alzheimer's Disease demographic and diagnosis information ##
ad_demo = pd.read_csv('Demographics/ad_demographics.csv')

ad_diag_all = pd.read_csv('Diagnoses/phecode_diagnoses/ad_diagnoses.csv')

# Only keep diagnoses mapped to phecodes that are organized into ICD-10 inspired chapters
ad_diag = ad_diag_all[~ad_diag_all['icd10_chapter'].isnull()]

## Get Control demographic and diagnosis information ##
con_demo = pd.read_csv('Demographics/con_demographics.csv')

con_diag_all = pd.read_csv('Diagnoses/phecode_diagnoses/con_diagnoses.csv')

# Only keep diagnoses mapped to phecodes that are organized into ICD-10 inspired chapters
con_diag = con_diag_all[~con_diag_all['icd10_chapter'].isnull()]

# Merge con_demo info to retain the remaining patients:
con_diag = con_demo['person_id'].to_frame().merge(con_diag,
                                                  how='left',
                                                  on='person_id')

In [ ]:
total_ad = ad_demo.shape[0] #Total alzheimer patients
total_con = con_demo.shape[0] #Total control patients

In [ ]:
total_ad

In [ ]:
total_con

### Only keep patients from MatchIt

AD patients

In [ ]:
# Get person_ids for MatchIt AD patients
ad_MatchIt = pd.read_csv('Demographics/RE_MI_ad_demo.csv')

In [ ]:
ad_diag = ad_diag[ad_diag['person_id'].isin(ad_MatchIt['person_id'])]

In [ ]:
# 422 patients each
ad_diag['person_id'].unique().shape

control patients

In [ ]:
# Get person_ids for MatchIt Control patients
con_MatchIt = pd.read_csv('Demographics/RE_MI_con_demo.csv')

In [ ]:
con_diag = con_diag[con_diag['person_id'].isin(con_MatchIt['person_id'])]

In [ ]:
# 844 patients each
con_diag['person_id'].unique().shape

### Only keep the following columns: 'person_id', 'PatientDurableKey', 'phecode', 'phenotype', 'icd10_chapter'

In [ ]:
ad_diag = ad_diag[['person_id', 
                   'PatientDurableKey',
                   'phecode',
                   'phenotype',
                   'icd10_chapter']].copy().drop_duplicates()
con_diag = con_diag[['person_id', 
                     'PatientDurableKey',
                     'phecode',
                     'phenotype',
                     'icd10_chapter']].copy().drop_duplicates()

AD patients' information

In [ ]:
ad_diag['phenotype'].value_counts()

In [ ]:
ad_diag.info()

control patients' information

In [ ]:
con_diag['phenotype'].value_counts()

In [ ]:
con_diag.info()

### Make pivot tables

In [ ]:
# Takes a few minutes
n = 'phenotype'
ad_diag_pivot = pd.pivot_table(ad_diag[[n, 'person_id']].drop_duplicates(), 
                               values=[n], 
                               index='person_id', 
                               columns=[n],
                               aggfunc=lambda x: 1 if len(x)>0 else 0, 
                               fill_value=0)
ad_diag_pivot['isalz'] = 1
ad_diag_pivot.head(3)
# Note: Each row is a patient, each column is diagnosis 
# Value is 1 if patient has diagnosis, 0 otherwise

In [ ]:
# 1688 patients
ad_diag_pivot.shape

In [ ]:
n = 'phenotype'
con_diag_pivot = pd.pivot_table(con_diag[[n, 'person_id']].drop_duplicates(), 
                                values=[n], 
                                index='person_id', 
                                columns=[n],
                                aggfunc=lambda x: 1 if len(x)>0 else 0, 
                                fill_value = 0)
con_diag_pivot['isalz'] = 0
con_diag_pivot.head(3)

In [ ]:
# 2049 control patients (1327 patients don't have any diagnosis)
con_diag_pivot.shape

In [ ]:
alldiag_pivot = pd.concat([ad_diag_pivot, con_diag_pivot], axis=0)

### Drop columns

In [ ]:
colstodrop = alldiag_pivot.columns[alldiag_pivot.columns.str.contains('alzheimer', 
                                                                      flags=re.IGNORECASE)]
colstodrop

In [ ]:
alldiag_pivot = alldiag_pivot.drop(colstodrop, axis=1)

### Make demographic df

In [ ]:
ad_demo.columns

In [ ]:
demographic_cols = ['person_id',
                    'estimated_age',
                    'gender_concept_id',
                    'race_concept_id',
                    'ethnicity_concept_id',
                    'UCSFDerivedRaceEthnicity_Clean',
                    'death_status',
                    'zip']
all_demo = pd.concat([ad_demo[demographic_cols], 
                      con_demo[demographic_cols]],
                      copy=False)

all_demo = all_demo.drop_duplicates().set_index('person_id').reindex(alldiag_pivot.index)

### Check that alldiag_pivot and all_demo dfs have the same number of rows, the same index, and fillna with 0.

In [ ]:
# Fill all NaN values with 0
alldiag_pivot = alldiag_pivot.fillna(0)
all_demo = all_demo.fillna(0)

In [ ]:
# Check the shape of both dfs
print('Shape of alldiag_pivot is {} and shape of all_demo is {}.'.format(alldiag_pivot.shape,
                                                                         all_demo.shape))

In [ ]:
# Check whether indices are the same for both dfs
pd.Series(alldiag_pivot.index == all_demo.index).unique()

In [ ]:
all_demo.head(3)

# Dimensionality Reduction

In [ ]:
y = alldiag_pivot['isalz'].replace({True:'Alzheimer',False:'Control'})
demographic_cols.remove('person_id')

In [ ]:
z = all_demo

In [ ]:
X = alldiag_pivot.drop('isalz', axis=1).astype('int32')

In [ ]:
%%time
mapper = umap.UMAP(metric='cosine', random_state=42, low_memory=True, verbose=1).fit(X)

In [ ]:
import pickle

filename = 'pickle_files/UMAP/AD_MI_phenotype_umap.model.pkl'

if os.path.isdir('pickle_files'):
    if os.path.isdir('pickle_files/UMAP'):
        # save the model to disk
        pickle.dump(mapper, open(filename, 'wb'))
    else:
        os.mkdir('pickle_files/UMAP')
        pickle.dump(mapper, open(filename, 'wb'))
else:
    os.mkdir('pickle_files')
    os.mkdir('pickle_files/UMAP')
    pickle.dump(mapper, open(filename, 'wb'))

# load file
mapper = pickle.load(open(filename, 'rb'))


In [ ]:
%%time
X_embedded = mapper.transform(X)

In [ ]:
# reset index for y
y = y.reset_index()
y = y['isalz']

In [ ]:
y = y.rename('Patients')
y = y.replace({1 : 'Alzheimer',
               0 : 'Control'})

In [ ]:
# Make Figures and Figures/UMAP folder if they don't exist already
if os.path.isdir('Figures'):
    if os.path.isdir('Figures/UMAP'):
        print('Figures/UMAP file path already exists.')
    else:
        print('Making UMAP directory...')
        os.mkdir('Figures/UMAP')
        print('Figures/UMAP file path created.')
else:
    print('Making Figures directory...')
    os.mkdir('Figures')
    print('Making UMAP directory...')
    os.mkdir('Figures/UMAP')
    print('Figures/UMAP file path created.')   

In [ ]:
savefigs = True

In [ ]:
with sns.color_palette("Set1"):
    fig = plt.figure(figsize=(10,8))
    indices = np.arange(X_embedded.shape[0])
    sns.scatterplot(x=X_embedded[indices ,0], 
                    y=X_embedded[indices ,1], 
                    hue=y[indices],
                    s=12, 
                    linewidth=.0, alpha=.6,
                    hue_order=['Alzheimer', 'Control'])
    ax = plt.gca()
    ax.set(xticks=[], yticks=[], facecolor='white')
    plt.title('Phenotypes as Features - UMAP')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    
    if savefigs:
        plt.savefig('Figures/UMAP/Fig_2A.pdf', filetype=
                    'pdf', 
                    dpi=300, 
                    bbox_inches='tight')

In [ ]:
from scipy.stats import mannwhitneyu
ADvals = X_embedded[y.values == 'Alzheimer',:]
convals = X_embedded[y.values == 'Control',:]
print('Axis 1: ',mannwhitneyu(ADvals[:,0], convals[:,0]))
print('Axis 2: ', mannwhitneyu(ADvals[:,1], convals[:,1]))

In [ ]:
with sns.axes_style("white"):
    with sns.color_palette("Set1"):
        plt.figure(figsize = (5,3))
        sns.violinplot(x=X_embedded[:,0], y=y, bw=.1)
        plt.xlabel('UMAP Component 1') 
        if savefigs:
            plt.savefig('Figures/UMAP/Fig_2B.pdf', 
                        filetype='pdf', 
                        dpi=300, 
                        bbox_inches='tight')
        
        plt.figure(figsize=(5,3))
        sns.violinplot(x=X_embedded[:,1], y=y, bw=.1)
        plt.xlabel('UMAP Component 2')
        if savefigs: 
            plt.savefig('Figures/UMAP/Fig_2C.pdf', 
                        filetype='pdf', 
                        dpi=300, 
                        bbox_inches='tight')

In [ ]:
# Change column name to Identitied Race and Ethnicity
z = z.rename({'UCSFDerivedRaceEthnicity_Clean' : 'Identified Race and Ethnicity'}, axis=1) 

In [ ]:
z.columns

In [ ]:
col = 'Identified Race and Ethnicity'

In [ ]:
# Unequal because of control patients not having a diagnosis
z[col].value_counts().sort_values()

In [ ]:
# control patients without an associated phecode and phenotype:
con_no_diag = con_diag[con_diag['phenotype'].isnull()]['person_id'].unique()

In [ ]:
con_demo[con_demo['person_id'].isin(con_no_diag)]['UCSFDerivedRaceEthnicity_Clean'].value_counts().sort_values()

Adding together should yield 1266 patients for each race/ethnicity

In [ ]:
z[col].value_counts().sort_values() + \
con_demo[con_demo['person_id'].isin(con_no_diag)]['UCSFDerivedRaceEthnicity_Clean'].value_counts().sort_values()

In [ ]:
# Change from Latinx to Latine for concordance with Romance Languages
z[col] = z[col].replace({'Latinx' : 'Latine'})

In [ ]:
z[col].reset_index()[col][indices].unique()

### Identified Race and Ethnicity

In [ ]:
savefigs = True

fig = plt.figure(figsize=(8,6))
sns.scatterplot(x=X_embedded[indices,0],
                y=X_embedded[indices,1], 
                hue=z[col].reset_index()[col][indices], 
                hue_order=['Asian', 
                           'Black or African American',
                           'Latine', 
                           'White or Caucasian'],
                s=12, 
                linewidth=0,
                alpha=.6,
                palette = 'Set2')
ax = plt.gca()
ax.set(xticks=[], yticks=[], facecolor='white')
plt.title('Race and Ethnicity', fontsize=12, fontweight='bold')
plt.legend(bbox_to_anchor=(-0.1, 1))

if True: 
    plt.savefig('Figures/UMAP/Fig_2D.pdf', bbox_inches='tight')
plt.show()

In [ ]:
with sns.color_palette("Set2"):
    val1 = X_embedded[z[col].values == 'Asian',:]
    val2 = X_embedded[z[col].values == 'Black or African American',:]
    val3 = X_embedded[z[col].values == 'Latine',:]
    val4 = X_embedded[z[col].values == 'White or Caucasian',:]
    
    
    sns.violinplot(x=X_embedded[:,0], 
                   y=z[col], 
                   bw=.1,
                   order=['Asian', 'Black or African American', 'Latine', 'White or Caucasian'])
    plt.xlabel('PC1')
    plt.xlabel("UMAP Component 1")
    
    if savefigs: 
        plt.savefig('Figures/UMAP/Fig_2E.pdf', 
                    filetype='png', 
                    dpi=300, 
                    bbox_inches='tight')
    
    plt.show()
    
    sns.violinplot(x=X_embedded[:,1], 
                   y=z[col], 
                   bw=.1,
                   order=['Asian', 
                          'Black or African American', 
                          'Latine', 
                          'White or Caucasian'])
    plt.xlabel('PC2')
    
    plt.xlabel("UMAP Component 2")
    
    if savefigs: 
        plt.savefig('Figures/UMAP/Fig_2F.pdf', 
                    filetype='png', 
                    dpi=300, 
                    bbox_inches='tight')
   
    plt.show()

### Saving identified race and ethnicity values for Dunn's test in R studio

In [ ]:
base_dir= os.getcwd()

In [ ]:
if not os.path.isdir(base_dir + "\\Tables"):
    print('Making Tables directory...')
    os.mkdir(base_dir + "\\Tables")
if not os.path.isdir(base_dir + "\\Tables\\UMAP"):
    print('Making UMAP directory...')
    os.mkdir(base_dir + "\\Tables\\UMAP")
    print('Making UMAP directory...')

In [ ]:
# Convert to pandas dataframes
A_RE = pd.DataFrame(val1)
B_RE = pd.DataFrame(val2)
L_RE = pd.DataFrame(val3)
W_RE = pd.DataFrame(val4)

In [ ]:
# Save as csv
for re_df, re_name in zip([A_RE, B_RE, L_RE, W_RE], ['Asian', 'Black', 'Latine', 'White']):
    re_df.to_csv(base_dir + '\\Tables\\UMAP\\UMAP_RE_UCSF_' + re_name + '.csv')